In [1]:
import os
import nltk
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords


In [2]:
label_list = ["CURRENT SMOKER", "NON-SMOKER", "PAST SMOKER", "UNKNOWN"]
train_list = os.listdir("train/")
contents = []
labels = []
for file_name in train_list:
    with open("train/"+file_name) as f:
        contents.append(str(f.read()))
    for i, label in enumerate(label_list):
        if label in file_name:
            labels.append(i)

In [3]:
seperate_contents = []
for content in contents:
    #print(content.split("\n")[:5])
    seperate_contents.append(content.split("\n")[5:])

from nltk.corpus import stopwords
nltk.download('punkt')
words = stopwords.words('english')
for w in ['!',',','.','?',':','/',';','(',')']:
    words.append(w)
# filtered = []


[nltk_data] Downloading package punkt to /home/tl32rodan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from nltk.tokenize import word_tokenize
for i,slide in enumerate(seperate_contents):
    seperate_contents[i] = word_tokenize(" ".join(slide))
filtered_contents = []
for i, pattern in enumerate(seperate_contents):
    filtered_contents.append([word for word in pattern if word not in words])

In [5]:
filtered_contents

[['1/9/2003',
  '12:00:00',
  'AM',
  'OB',
  'Discharge',
  'Summary',
  'Signed',
  'DIS',
  'Report',
  'Status',
  'Signed',
  'OB',
  'EMR',
  'L',
  'D',
  'DISCHARGE',
  'SUMMARY',
  'NAME',
  'ADEIDRE',
  'AUJERIELLE',
  'WHIPPLE',
  'UNIT',
  'NUMBER',
  '886-63-46',
  'ADMISSION',
  'DATE',
  '20030109',
  'DISCHARGE',
  'DATE',
  '20030113',
  'PRINCIPAL',
  'DISCHARGE',
  'DIAGNOSIS',
  'Vaginal',
  'Delivery',
  'With',
  'First',
  'Degree',
  'Laceration',
  'ASSOCIATED',
  'DIAGNOSES',
  'Advanced',
  'Maternal',
  'Age',
  'Anxiety',
  'Disorder',
  'PRINCIPAL',
  'PROCEDURE',
  'OR',
  'OPERATION',
  'Spontaneous',
  'Vertex',
  'Vaginal',
  'Delivery',
  'ASSOCIATED',
  'PROCEDURES',
  'OR',
  'OPERATIONS',
  'POSTPARTUM',
  'DIAGNOSTIC',
  'PROCEDURES',
  'None',
  'POSTPARTUM',
  'THERAPEUTIC',
  'PROCEDURES',
  'None',
  'HISTORY',
  'AND',
  'REASON',
  'FOR',
  'HOSPITALIZATION',
  'Active',
  'Labor',
  'PHYSICAL',
  'EXAMINATION',
  'HEIGHT',
  'NORMAL',
  '70

In [6]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk


tagged_data = [TaggedDocument(words=" ".join(_d), tags=[labels[i]]) for i, _d in enumerate(filtered_contents)]

In [7]:
def train_doc2vec_model(tagged_data, max_epochs = 100, vec_size = 10, alpha = 0.025):

    model = Doc2Vec(size=vec_size,
                    alpha=alpha,
                    min_alpha=0.00025,
                    min_count=1,
                    dm=1)

    '''
    Note: dm defines the training algorithm. If dm=1 means‘distributed memory’(PV-DM) and dm =0 means‘distributed bag of words’(PV-DBOW). 
    Distributed Memory model preserves the word order in a document whereas Distributed Bag of words just uses the bag of words approach, 
    which doesn’t preserve any word order.
    '''

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    #model.save("d2v_"+str(vec_size)+"_flitered.model")
    model.save("d2v_"+str(vec_size)+".model")
    print("Model Saved: ","d2v_"+str(vec_size)+".model")


In [9]:
# -------------Lets start training our model----------------------
max_epochs = 100
#vec_size = 10
alpha = 0.025

#for vec_size in [10,30,300,7000]:
for vec_size in [300]:
    train_doc2vec_model(tagged_data, max_epochs = max_epochs, vec_size = vec_size, alpha = alpha)

iteration 0
iteration 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteration 78
iterati

## Define classifier model & training, testing functions

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class MLP(nn.Module):
    def __init__(self, in_dim=7000, hidden_size=1024 ,num_classes=4):
        super(MLP, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(in_dim,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,num_classes)
        )
    def forward(self, x):
        output = self.model(x)
        return output

In [9]:
def train(mlp, train_data, labels, optimizer, epochs=1000):

    criterion = nn.CrossEntropyLoss()
    
    for epochs in range(epochs):
        optimizer.zero_grad()
        output = mlp(train_x)
        loss = criterion(output, train_labels)
        loss.backward()
        optimizer.step()
        
    

In [10]:
def test(output, labels):
    if type(output) == torch.Tensor:
        predicted = output.data.cpu().numpy()
        predicted = np.argmax(predicted, axis=1)
    else:
        predicted = np.array(output)

    cnt=0
    for i, x in enumerate(predicted):
        if(x==labels[i]):
            cnt+=1
    print('Accuracy: ',cnt/len(labels)*100.0)

## vec_size=7000

In [11]:
# -------------Lets play with it----------------------
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec.load("d2v_7000.model")
train_data = []
for content in seperate_contents:
    train_data.append(model.infer_vector(content))

    
train_x = torch.tensor(train_data)
train_labels = torch.tensor(labels)
train_x

tensor([[-4.2098e-03,  1.4922e-02, -3.1769e-02,  ...,  5.5077e-03,
          7.6766e-04, -1.1845e-04],
        [-1.2210e-02,  2.8193e-02, -6.7502e-02,  ...,  1.8829e-02,
          9.0297e-03, -3.4946e-03],
        [-3.3442e-02,  7.4840e-02, -1.4073e-01,  ...,  2.8296e-02,
          2.2566e-03,  5.7149e-03],
        ...,
        [ 2.1589e-02,  2.8095e-02, -9.1812e-02,  ...,  2.1096e-02,
          5.1214e-03, -1.5829e-02],
        [-1.5302e-02,  3.3168e-02, -7.5407e-02,  ...,  2.2468e-02,
          1.0507e-02, -1.6738e-03],
        [-1.6678e-02,  4.1065e-02, -9.0989e-02,  ...,  1.8303e-02,
          2.0563e-03, -2.3405e-03]])

In [12]:
mlp = MLP(7000, 1024, len(label_list))

optimizer = optim.SGD(mlp.parameters(), lr=0.01, weight_decay=5e-3, momentum=0.9)
train(mlp, train_data, labels, optimizer)

In [13]:
output = mlp(train_x)
test(output, labels)

Accuracy:  75.0


## vec_size=300

In [ ]:
# -------------Lets play with it----------------------
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec.load("d2v_300.model")
train_data = []
for content in seperate_contents:
    train_data.append(model.infer_vector(content))

train_x = torch.tensor(train_data)
train_labels = torch.tensor(labels)


In [ ]:
mlp = MLP(300, 100, len(label_list))
optimizer = optim.SGD(mlp.parameters(), lr=0.05, weight_decay=5e-3, momentum=0.9)
train(mlp, train_data, labels, optimizer)

In [ ]:
output = mlp(train_x)
test(output, labels)

## vec_size=30

In [ ]:
# -------------Lets play with it----------------------
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec.load("d2v_30.model")
train_data = []
for content in seperate_contents:
    train_data.append(model.infer_vector(content))

train_x = torch.tensor(train_data)
train_labels = torch.tensor(labels)

mlp = MLP(30, 10, len(label_list))
optimizer = optim.SGD(mlp.parameters(), lr=0.1)
train(mlp, train_data, labels, optimizer)


In [ ]:
output = mlp(train_x)
test(output, labels)

## vec_size=10

In [ ]:

# -------------Lets play with it----------------------
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec.load("d2v_10.model")
train_data = []
for content in seperate_contents:
    train_data.append(model.infer_vector(content))

train_x = torch.tensor(train_data)
train_labels = torch.tensor(labels)

mlp = MLP(10, 10, len(label_list))
optimizer = optim.SGD(mlp.parameters(), lr=0.01)
train(mlp, train_data, labels, optimizer)


In [ ]:
output = mlp(train_x)
test(output, labels)

## Seperate training & testing data

In [92]:

model = Doc2Vec.load("d2v_7000.model")
data = []
for content in seperate_contents:
    data.append(model.infer_vector(content))


### Train with MLP

In [107]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.33, random_state=5)

In [108]:
train_labels

[1, 3, 2, 3, 1, 3, 3, 1, 3, 1, 2, 0, 2, 2, 1, 0, 0, 3, 0, 1, 0, 2, 0, 3, 1, 0]

In [109]:

train_x = torch.tensor(train_x)
train_labels = torch.tensor(train_labels)

test_x = torch.tensor(test_x)
test_labels = torch.tensor(test_labels)

In [110]:
mlp = MLP(7000, 1280, len(label_list))
optimizer = optim.SGD(mlp.parameters(), lr=0.05, weight_decay=5e-3, momentum=0.9)

train(mlp, train_x, train_labels, optimizer, epochs=100)

In [111]:
output = mlp(test_x)
test(output, test_labels)

Accuracy:  21.428571428571427


In [112]:
output = mlp(train_x)
test(output, train_labels)

Accuracy:  65.38461538461539


In [113]:
test_labels

tensor([2, 3, 2, 0, 0, 0, 3, 2, 1, 2, 3, 1, 2, 1])

### Train with SVM classifier

In [114]:

model = Doc2Vec.load("d2v_7000.model")
data = []
for content in seperate_contents:
    data.append(model.infer_vector(content))

In [115]:
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=0, tol=1e-5)

In [128]:
train_x, test_x, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.33, random_state=15)

In [129]:
model.fit(train_x, train_labels)

LinearSVC(random_state=0, tol=1e-05)

In [130]:
output = model.predict(train_x)
test(output, train_labels)

Accuracy:  69.23076923076923


In [131]:
output = model.predict(test_x)
test(output, test_labels)

Accuracy:  57.14285714285714
